# Native Dev Framework Support for Foundation Models

## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also [create a documentation project](https://docs.validmind.ai/guide/create-your-first-documentation-project.html) on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [3]:
%pip install validmind


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

In a browser, go to the **Client Integration** page of your documentation project and click **Copy to clipboard** next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

::: {.column-margin}
::: {.callout-tip}
This step requires a documentation project. [Learn how you can create one](https://docs.validmind.ai/guide/create-your-first-documentation-project.html).
:::
:::

Next, replace this placeholder with your own code snippet:

In [4]:
## Replace the code below with the code snippet from your project ## 

import validmind as vm

vm.init(
    api_host="http://localhost:3000/api/v1/tracking",
    project="clkh0yg4r00cgg9ryfwfy8pqo",
)

2023-09-06 11:44:25,748 - INFO(validmind.api_client): Connected to ValidMind. Project: Demo project 2 (clkh0yg4r00cgg9ryfwfy8pqo)


In [5]:
vm.preview_template()

Accordion(children=(HTML(value='<p>Empty Section</p>'),), titles=("1. Model Validation ('model_validation')",)…

### Download test dataset
https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news
Download the dataset in the above link and move it into the current directory.

In [6]:
from validmind.models import FoundationModel, Prompt

In [7]:
import os

import dotenv
dotenv.load_dotenv()

if os.getenv("OPENAI_API_KEY") is None:
    raise Exception("OPENAI_API_KEY not found")

In [8]:
import openai

def call_model(prompt):
    return openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    ).choices[0].message["content"]

In [9]:
prompt_template = """
You are an AI with expertise in sentiment analysis, particularly in the context of financial news.
Your task is to analyze the sentiment of a specific sentence provided below.
Before proceeding, take a moment to understand the context and nuances of the financial terminology used in the sentence.

Sentence to Analyze:
```
{Sentence}
```

Please respond with the sentiment of the sentence denoted by one of either 'positive', 'negative', or 'neutral'.
Please respond only with the sentiment enum value. Do not include any other text in your response.

Note: Ensure that your analysis is based on the content of the sentence and not on external information or assumptions.
""".strip()

prompt_variables = ["Sentence"]

In [10]:
import pandas as pd

df = pd.read_csv('./datasets/sentiments.csv')

df_test = df[:10].reset_index(drop=True)
df_test

,Sentiment,Sentence
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
5,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...
6,positive,"For the last quarter of 2010 , Componenta 's n..."
7,positive,"In the third quarter of 2010 , net sales incre..."
8,positive,Operating profit rose to EUR 13.1 mn from EUR ...
9,positive,"Operating profit totalled EUR 21.1 mn , up fro..."


In [11]:
vm_dataset = vm.init_dataset(
    dataset=df,
    text_column="Sentence",
    target_column="Sentiment",
)

vm_test_ds = vm.init_dataset(
    dataset=df_test,
    text_column="Sentence",
    target_column="Sentiment",
)

vm_model = FoundationModel(
    predict_fn=call_model,
    prompt=Prompt(
        template=prompt_template,
        variables=prompt_variables,
    ),
    train_ds=vm_test_ds,
    test_ds=vm_test_ds,
)

2023-09-06 11:44:25,903 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-09-06 11:44:25,921 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-09-06 11:44:25,927 - INFO(validmind.models.foundation): Running predict() for `train_ds`... This may take a while
2023-09-06 11:44:32,651 - INFO(validmind.models.foundation): Running predict() for `test_ds`... This may take a while


In [12]:
from validmind.vm_models import TestContext

test_context = TestContext(model=vm_model)

In [13]:
from IPython.display import display

from validmind.tests import load_test

In [14]:
test_ids = [
    "validmind.prompt_validation.Specificity",
    "validmind.prompt_validation.Clarity",
    "validmind.prompt_validation.Conciseness",
    "validmind.prompt_validation.NegativeInstruction",
    "validmind.prompt_validation.Escape",
    "validmind.prompt_validation.Robustness",
]
tests = []

for test_id in test_ids:
    Test = load_test(test_id)

    test = Test(test_context, params={})
    tests.append(test)

    test.run()

    display(test.result.show())

None

None

None

None

None

AttributeError: 'str' object has no attribute 'iterrows'

In [ ]:
vm.run_documentation_tests(model=vm_model)

2023-09-06 09:47:03,577 - ERROR(validmind.tests): Unable to load test nonexistent_provider.test. No Test Provider found for the namespace: nonexistent_provider.
2023-09-06 09:47:03,578 - ERROR(validmind.tests): Unable to load test validmind.nonexistent_test. No module named 'validmind.tests.'


2023-09-06 09:47:03,582 - ERROR(validmind.tests): Unable to load test nonexistent_provider.test. No Test Provider found for the namespace: nonexistent_provider.
2023-09-06 09:47:03,582 - ERROR(validmind.tests): Unable to load test validmind.nonexistent_test. No module named 'validmind.tests.'
2023-09-06 09:47:03,583 - ERROR(validmind.vm_models.test_plan): Failed to run test 'class_imbalance': (MissingRequiredTestContextError) Dataset 'dataset' is a required input and must be passed as a keyword argument to the test plan
2023-09-06 09:47:03,584 - ERROR(validmind.vm_models.test_plan): Failed to run test 'duplicates': (MissingRequiredTestContextError) Dataset 'dataset' is a required input and must be passed as a keyword argument to the test plan
2023-09-06 09:47:03,584 - ERROR(validmind.vm_models.test_plan): Failed to run test 'cardinality': (MissingRequiredTestContextError) Dataset 'dataset' is a required input and must be passed as a keyword argument to the test plan
2023-09-06 09:47:03

BinaryClassificationTestSuite(config=None, _global_config=None, _test_configs=None, test_context=TestContext(dataset=None, model=<validmind.models.foundation.FoundationModel object at 0x16b8933d0>, models=None, context_data=None), fail_fast=False, _tests=None, dataset=None, model=<validmind.models.foundation.FoundationModel object at 0x16b8933d0>, models=None, pbar=IntProgress(value=55, max=58), pbar_description=Label(value='Test suite complete!'), pbar_box=HBox(children=(Label(value='Test suite complete!'), IntProgress(value=55, max=58))), summary=None, _test_plan_classes=[<class 'validmind.template.DataPreparationTestPlan'>, <class 'validmind.template.ModelDevelopmentTestPlan'>], _test_plan_instances=[DataPreparationTestPlan(test_context=TestContext(dataset=None, model=<validmind.models.foundation.FoundationModel object at 0x16b8933d0>, models=None, context_data=None)), ModelDevelopmentTestPlan(test_context=TestContext(dataset=None, model=<validmind.models.foundation.FoundationModel 